<a href="https://colab.research.google.com/github/R-802/LING-226-Assignments/blob/main/Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LING226 Assignment Two
- Shemaiah Rangitaawa
- `300601546`

**Research Question:** "How does the use of descriptive language differ in bestselling mystery novels compared to bestselling science fiction novels, and what does this tell us about the role of setting and atmosphere in genre-specific storytelling?"